# Tutorial 6: Solving an ODE

In [4]:
# Load packages:

# this package allows to work efficiently with arrays
import numpy as np
# this package provides some tools of linear algebra. We can use them to simplify certain algorithms and for comparisons.
import scipy.linalg as lin
# this package is used to draw graphs
import matplotlib.pyplot as plt

The aim of this tutorial is to compare different numerical methods for solving a vectorial ODE 
$$
\frac{dU}{dt} = AU + S
$$ 
where $A \in \mathbb{R}^{N \times N}$ and $S \in \mathbb{R}^N$. 

We study numerical methods to solve this system.

---

## An iterative method: Jacobi algorithm

Consider the problem 

$$MU=b$$

where $M\in\mathbb{R}^{N\times N}$ and $U,b\in\mathbb{R}^N$. Decompose 

$$M = D - R$$

where $D =  Diag(M_{1,1},\dots,M_{N,N})$ is the diagonal of $M$ and $R = D-M$ is the remaining part.

Define a sequence $(V^k)_{k\in\mathbb{N}}$ by chosing an initial $V^0 \in\mathbb{R}^N$ and then iteratively 

$$D V^{k+1} = R V^k + b.$$ 

1) Assuming that $M_{i,i} \neq 0$, verify that 
- the sequence $(V^k)_{k\in\mathbb{N}}$ is well-defined, 
- if it converges, then its limit solves $MU = b$, i.e. $\lim\limits_{k\rightarrow \infty}V^k = V^\infty = U$. 
- What information is added to this limit when you consider furthermore that the whole matrix $M = D-R$ is invertible?

**Answer:**

- $V^{k+1} = D^{-1} (R V^k +b)$. Since $D = Diag(M_{1,1},\dots,M_{N,N})$ with $M_{i,i} \neq 0$, then $D$ is invertible and $V^{k+1}$ is well-defined.
- If convergence occurs, $D V^{\infty} = R V^{\infty} + b$ then $(D-R) V^\infty = M V^\infty = b$ is a solution to the problem.
- If $M$ is invertible, then the problem has a unique solution $M^{-1} b$.

2) What else could happen to $(V^k)$ in the limit $k\rightarrow + \infty$? 

**Answer:**

The sequence could diverge.

3) a) In the test below, we will use the parameters 

$$ A = \left( \begin{array}{ccc} 3 & 1 & 0 \\ -1 & -5 & 1 \\ 0 & 2 & 4 \end{array} \right), \qquad b = \left( 4, -5, 6\right)^T. $$

Compute the solution $V$ of the problem $AV = b$. 

b) Implement a function computing iteratively $V^{k+1}$ for solving $MU = b$. Stop the algorithm at iteration $k$ if 
- $k>k_{max}$, too many iterations are performed.
- or $\epsilon^k = {\|AV^k-b\|} < \epsilon_{max}$, a desired accuracy is obtained.  

The function should 
- take $M$, $b$, $V^0$, a maximum number of iteration $k_{\max}$ and a certain tolerance $\epsilon_{max}$ for arguments.
- return $V^k$, the final solution at last $k$, the vector of the errors $(\epsilon^i)_{i=1,\dots,k}$ and the final $k$ (either $k$ is such that the desired accuracy is reached or $k=k_{max}$).

c) Test your algorithm with the parameters in a) and with $\epsilon_{max} = 10^{-6}$ and $k_{max}=100$. Compare with different values of the inital vector $V^0$. 

**Answer:**

a) $V = (4, -5, 6)^T$

In [5]:
#b)
def iterative_solver(M, b, Vinit, kmax, epsmax):
    """
    Provides an approximation of M^{-1}b using the proposed iterative method
    ----------   
    parameters:
    M      : matrix in the equation MV = b to solve (numpy array of size N,N)
    b      : vector on the right-hand-side of the equation MV = b (numpy array of size N)
    Vinit  : initial vector of the iterative method (numpy array of size N)
    kmax   : maximum number of iterations, stops if k reaches kmax (integer)
    epsmax : tolerance on the residual, stops if eps reaches epsmax (float)
    
    returns:
    V   : resulting vector at the end of the iterations (numpy array of size N)
    eps : vector composed of the residuals eps at every iteration (numpy array of size k)
    k   : number of iterations performed before the algorithm stops (integer)
    """

    N      = len(b)
    V      = np.copy(Vinit)
    V_new  = np.copy(Vinit)
    eps    = np.zeros(kmax)
    eps[0] = lin.norm(np.matmul(M,V_new)-b)
    
    k     = 0
    while (k<kmax-1) and (eps[k] > epsmax):   
        k += 1
        V = np.copy(V_new)
        for i in range(N):
            V_new[i]  = sum( M[i,j]*V[j] for j in range(i    ) )
            V_new[i] += sum( M[i,j]*V[j] for j in range(i+1,N) )
            V_new[i] += -b[i]
            V_new[i] /= -M[i,i]
        eps[k] = lin.norm(np.matmul(M,V_new)-b)
    return V_new, eps[:k+1], k

In [6]:
#c)
A = np.array([[3.,1,0],[-1,-5,1],[0,2,4]])
b = np.array([4.,-5,6])
V, eps, k = iterative_solver(B, b, b, 100, 1.e-6)

print("Solution V = ", V)
print("AV         = ", np.matmul(B,V))
print("b          = ",b)

NameError: name 'B' is not defined

4) Consider the matrices $B(x) \in\mathbb{R}^{N\times N}$ defined for $x\in\mathbb{R}$ by: 

$$B(x)_{i,j} = x \delta_{i,j} - \frac{1}{N}.$$

Choose the data: 

$$N = 10,  \quad \epsilon_{max} = 10^{-8}, \quad b \in\mathbb{R}^N \quad\text{and}\quad V^0 \in\mathbb{R}^N \quad\text{s.t.}\quad b_i=1, \quad V^0 = e^0,$$

and the matrix to invert $M=B(x=2)$.

a) Compute $V^k$ and $MV^k - b$ at the end of the iterations. 

b) Verify that the error $\epsilon^k$ is inferior to desired accuracy $\epsilon_{max}$ or that $k=100$.

c) Plot the error $\epsilon^k$ as a function of $k$ in logscale.  

In [ ]:
### test your aglorithm here
# a) and b)

#Construction of the matrices and vectors
N = 10
x = 2.

M        = x * np.eye(N) - np.outer(np.ones(N), np.ones(N)) / N 
b        = np.ones(N)
Vinit    = np.zeros(N); Vinit[0] = 1.

#Parameter of the method
kmax     = 100
epsmax   = 1.e-8

U, eps, k = iterative_solver(M, b, Vinit, kmax, epsmax)

print('solution = ', U)
print('final residual = ', np.matmul(M,U)-b)
print('final iteration', k)

In [ ]:
# c)
# Plot the errors
vect_err = eps
vect_k   = range(len(eps))

plt.figure(1)
plt.semilogy(vect_k,vect_err)
plt.xlabel('k')
plt.ylabel('Error')
plt.title("Error(k)")
plt.show()

5) What can you say about the convergence rate of the algorithm, i.e. the speed of convergence of the sequence $(V^k)_{k\in\mathbb{N}}$ to the desired result?

**Answer:**

It is of the form $\rho(M)^k$.

6) Reproduce the previous computations with $x=5$ and $x=10$, and plot on the same graph the errors obtained with your algorithm for the different values of $x$. Interprete the evolution of this convergence rate with $x$.

In [ ]:
#Construction of the matrices and vectors
N = 10
x = 5.

M2       = x * np.eye(N) - np.outer(np.ones(N), np.ones(N)) / N 
b        = np.ones(N)
Vinit    = np.zeros(N); Vinit[0] = 1.

#Parameter of the method
kmax     = 100
epsmax   = 10**(-8)

# Compute the solution and the errors
U2, eps2, k2 = iterative_solver(M2, b, Vinit, kmax, epsmax)

vect_err2 = eps2
vect_k2   = range(len(eps2))

print('solution2 = ', U2)
print('final residual = ', np.matmul(M2,U2)-b)
print('final iteration2', k2)

In [ ]:
#Construction of the matrices and vectors
N = 10
x = 10.

M3       = x * np.eye(N) - np.outer(np.ones(N), np.ones(N)) / N 
b        = np.ones(N)
Vinit    = np.zeros(N); Vinit[0] = 1.

#Parameter of the method
kmax     = 100
epsmax   = 10**(-8)

# Compute the solution and the errors
U3, eps3, k3 = iterative_solver(M3, b, Vinit, kmax, epsmax)

vect_err3 = eps3
vect_k3   = range(len(eps3))

print('solution2 = ', U3)
print('final residual = ', np.matmul(M3,U3)-b)
print('final iteration2', k3)

In [ ]:
# Plot the different errors 
plt.figure(2)
plt.semilogy(vect_k , vect_err , 'r', label='x=2' )
plt.semilogy(vect_k2, vect_err2, 'b', label='x=5' )
plt.semilogy(vect_k3, vect_err3, 'g', label='x=10')
plt.xlabel('k')
plt.ylabel('Error')
plt.legend()
plt.title("Error(k)")
plt.show()

7) Test again the algorithm with $x=\frac{2}{N}$. Give an interpretation.

In [ ]:
#Construction of the matrices and vectors
N = 10
x = 2./N

M4       = x * np.eye(N) - np.outer(np.ones(N), np.ones(N)) / N 
b        = np.ones(N)
Vinit    = np.zeros(N); Vinit[0] = 1.

#Parameter of the method
kmax     = 100
epsmax   = 10**(-8)

# Compute the solution and the errors
U4, eps4, k4 = iterative_solver(M4, b, Vinit, kmax, epsmax)

vect_err4 = eps4
vect_k4   = range(len(eps4))

print('solution2 = ', U4)
print('final residual = ', np.matmul(M4,U4)-b)
print('final iteration2', k4)

In [ ]:
plt.figure(4)
plt.semilogy(vect_k4, vect_err4, 'r', label='x=2/N')
plt.xlabel('k')
plt.ylabel('Error')
plt.legend()
plt.title("Error(k)")
plt.show()

**Answer:**

The spectral radius is bigger than 1, it diverges.

---

## Application for solving the ODE

In order to solve this ODE numerically, we need to reduce it into a finite dimensional problem. For this purpose, we use an implicit Euler time discretization: This method consists in approximating the value of $U$ at a finite number of values $(t^n)_{n=0,\dots,N}$. One writes 

$$ \frac{U^{n+1} - U^n}{\Delta t} = A U^{n+1} + S, \qquad{} (1)$$

where $U^n \approx U(t^n)$ approximates the value of $U$ at time $t^n$, and $\Delta t = t^{n+1}-t^n$ is the time step between the times $t^{n+1}$ and $t^n$. We choose to fix $\Delta t$ constant for all $n$. Remark that this definition of $U^{n+1}$ is equivalent to the vectorial equation of the last question, but without the infinitesimal hypothesis of $\delta t$. This is therefore only an approximation. 

1) Rewrite the discrete equation (1) under the form $B U^{n+1} = C U^n + E S$ (write down the matrices $B$, $C$ and $E$). 

**Answer:**

One now has 

$$ (Id - \Delta t A)U^{n+1} = U^n + \Delta t S.$$ 

As a result, $B = Id - \Delta t A$, $C = Id$ and $E = \Delta t Id$.

2) Using Gershgorin theorem:

a) What can you deduce on the eigenvalues of $B$ when $\Delta t$ is small? 

b) What can you deduce on the eigenvalues of its Jacobi iteration matrix ?

**Answer:**

a) The Gershgorin discs are $$D_i = B\left(1-\Delta t A_{i,i}, \quad{} \Delta t \sum\limits_{j\neq i} |A_{i,j}|\right),$$ therefore for $\Delta t$ small enough, these are bigger than 0.

b) The Gershgorin discs are $$D_i = B\left(0, \quad{}\frac{\Delta t \sum\limits_{j\neq i} |A_{i,j}|}{1-\Delta t A_{i,i}}\right),$$ therefore for $\Delta t$ small enough, these are smaller than 0 in norm.

3) Implement a function that 
- take as arguments: initial vector $U_0$, the matrix $A$, the source vector $S$ (constant), the time step $\Delta t$, the final time $T$ and the maximum number of iteration $k_{\max}$ and maximum tolerance $\epsilon_{\max}$
of the iterative mathod
- returns the sequence of the approximate solution $U^{n}$.  

It should compute iteratively each $U^{n+1}$ based on $U^n$, by solving the linear system (2) with the Jacobi iteration method of the last section. Choose $U^n$ as an initialisation when using this method to compute $U^{n+1}$.

In [ ]:
def solve_ODE_Jacobi(U0, A, S, dt, T_end, kmax, epsmax):
    """
    Computes the sequence U^n defined iteratively in (1), where every U^{n+1} is computed from U^n by using "iterative_solver" 
    ----------   
    parameters:
    U0     : first value of the iterative sequence U^n (numpy array of size N)
    A      : matrix on the right-hand-side of (1) (numpy array of size N,N)
    S      : vector of sources (independent of n) on the right-hand-side of (1) (numpy array of size N)
    dt     : fixed time step (float)
    T_end  : final time (float ; proportional to dt -> n_time * dt)
    kmax   : maximum number of iterations, stops if k reaches kmax (integer)
    epsmax : tolerance on the residual, stops if eps reaches epsmax (float)
    
    returns:
    U      : sequence of solutions U^n (numpy array of size N,n_time)
    """
        
    N       = len(S)
    n_final = int(T_end/dt)
    
    #initialization
    U          = np.zeros((N,n_final))
    U[:,0]     = U0
    iterations = np.zeros(n_final)
    C   = np.eye(N) - dt * A
            
    # time loop
    t = 0
    for n in range(n_final-1):    
        t    += dt       
        Vinit = U[:,n]
        b     = Vinit + dt * S
        U[:,n+1], _, iterations[n+1] = iterative_solver(C, b, Vinit, kmax, epsmax)
        
    return U

4) Test it with the matrix $A$ such that $A_{i,j} = 1/N$ for all $i,j$.

- Problem parameters: Choose the data: $N = 10$, $S \in\mathbb{R}^N$ such that $S=(1,2,\dots,N)$. 
- ODE parameters: Choose $U^0 = (N,\dots, 2,1)$ at initial time, final time is $T=1$ and fix $\Delta t = \frac{1}{100}$ (or equivalently do $n_{final} = 100$ time steps).
- Iterative methods parameters: Choose the maximum error is $\epsilon_{max} = 10^{-8}$, and a maximum number of iteration $k_{max} = 100$.

Plot the solution $U_i^{n_{final}}$ on a graph as a function of $i$. 

In [ ]:
###Test your algorithm here

#Problem parameters
N   = 10

#ODE parameters
dt    = 1/100.
T_end = 1.

U0    = np.array([N-i for i in range(N)])
S     = np.array([i+1 for i in range(N)]) 
A     = np.ones((N,N)) / 10

#Iterative method parameters
epsmax = 10**(-8)
kmax   = 100

# Solve
U = solve_ODE_Jacobi(U0, A, S, dt, T_end, kmax, epsmax)
n_times = len(U[0,:])

# Plot
plt.figure(3)
plt.xlabel('t')
plt.ylabel('U_i')
plt.title('U_i(t)')
for i in range(len(U[:,0])):
    plt.plot(range(n_times),U[i,:],label=i)
plt.legend()
plt.show()